# Getting Started With Text Embeddings

#### Project environment setup

- Load credentials and relevant Python Libraries
- If you were running this notebook locally, you would first install Vertex AI.  In this classroom, this is already installed.
```Python
!pip install google-cloud-aiplatform
```


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")

#### Enter project details

#### Use the embeddings model
- Import and load the model.

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko")

- Generate a word embedding

In [ ]:
embedding = embedding_model.get_embeddings(
    ["life"])

- The returned object is a list with a single `TextEmbedding` object.
- The `TextEmbedding.values` field stores the embeddings in a Python list.

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

- Generate a sentence embedding.

In [ ]:
embedding = embedding_model.get_embeddings(
    ["What is the meaning of life?"])

In [ ]:
embedding

In [ ]:
vector = embedding[0].values
print(f"Length = {len(vector)}")
print(vector[:10])

#### Similarity

- Calculate the similarity between two sentences as a number between 0 and 1.
- Try out your own sentences and check if the similarity calculations match your intuition.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
emb_1 = embedding_model.get_embeddings(
    ["What is the meaning of life?"]) # 42!

emb_2 = embedding_model.get_embeddings(
    ["How does one spend their time well on Earth?"])

emb_3 = embedding_model.get_embeddings(
    ["Would you like a salad?"])

vec_1 = [emb_1[0].values]
vec_2 = [emb_2[0].values]
vec_3 = [emb_3[0].values]

- Note: the reason we wrap the embeddings (a Python list) in another list is because the `cosine_similarity` function expects either a 2D numpy array or a list of lists.
```Python
vec_1 = [emb_1[0].values]
```

In [ ]:
print(cosine_similarity(vec_1,vec_2)) 
print(cosine_similarity(vec_2,vec_3))
print(cosine_similarity(vec_1,vec_3))

#### From word to sentence embeddings
- One possible way to calculate sentence embeddings from word embeddings is to take the average of the word embeddings.
- This ignores word order and context, so two sentences with different meanings, but the same set of words will end up with the same sentence embedding.

In [ ]:
in_1 = "The kids play in the park."
in_2 = "The play was for kids in the park."

- Remove stop words like ["the", "in", "for", "an", "is"] and punctuation.

In [ ]:
in_pp_1 = ["kids", "play", "park"]
in_pp_2 = ["play", "kids", "park"]

- Generate one embedding for each word.  So this is a list of three lists.

In [ ]:
embeddings_1 = [emb.values for emb in embedding_model.get_embeddings(in_pp_1)]

- Use numpy to convert this list of lists into a 2D array of 3 rows and 768 columns.

In [ ]:
import numpy as np
emb_array_1 = np.stack(embeddings_1)
print(emb_array_1.shape)

In [ ]:
embeddings_2 = [emb.values for emb in embedding_model.get_embeddings(in_pp_2)]
emb_array_2 = np.stack(embeddings_2)
print(emb_array_2.shape)

- Take the average embedding across the 3 word embeddings 
- You'll get a single embedding of length 768.

In [ ]:
emb_1_mean = emb_array_1.mean(axis = 0) 
print(emb_1_mean.shape)

In [ ]:
emb_2_mean = emb_array_2.mean(axis = 0)

- Check to see that taking an average of word embeddings results in two sentence embeddings that are identical.

In [ ]:
print(emb_1_mean[:4])
print(emb_2_mean[:4])

#### Get sentence embeddings from the model.
- These sentence embeddings account for word order and context.
- Verify that the sentence embeddings are not the same.

In [ ]:
print(in_1)
print(in_2)

In [ ]:
embedding_1 = embedding_model.get_embeddings([in_1])
embedding_2 = embedding_model.get_embeddings([in_2])

In [ ]:
vector_1 = embedding_1[0].values
print(vector_1[:4])
vector_2 = embedding_2[0].values
print(vector_2[:4])

# Visualizing Embeddings

#### Project environment setup

- Load credentials and relevant Python Libraries

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")

## Embeddings capture meaning

In [ ]:
in_1 = "Missing flamingo discovered at swimming pool"

in_2 = "Sea otter spotted on surfboard by beach"

in_3 = "Baby panda enjoys boat ride"


in_4 = "Breakfast themed food truck beloved by all!"

in_5 = "New curry restaurant aims to please!"


in_6 = "Python developers are wonderful people"

in_7 = "TypeScript, C++ or Java? All are great!"


input_text_lst_news = [in_1, in_2, in_3, in_4, in_5, in_6, in_7]

In [ ]:
in_1 = "These apples are always crisp and delicious! My family loves them."
in_2 = "I bought these apples, and they were mushy and tasted bad. Not happy at all."
in_3 = "The cereal selection here is fantastic. I can always find my favorite brands."
in_4 = "The cereal I bought was stale. It was like eating cardboard."

in_5 = "Just discovered the new organic section at my local grocery store! 🌿 So excited to shop for healthier options. Thanks, @GreenGroceryCo! #HealthyEating #OrganicFood"
in_6 = "Visited @SuperMart today for my weekly fruit haul. Their produce section is always on point! 🍎🥭🍇 #FreshProduce #SuperMartLove"
in_7 = "Seriously disappointed with my recent trip to @BudgetMart. Out of stock on half the items I needed. 😡 #PoorService #BudgetMartFail"

in_8 = "In a recent survey of 1,000 consumers, 72% indicated a growing preference for organic and locally sourced products when shopping for groceries."
in_9 = "Our market research shows that plant-based meat alternatives have seen a 35% increase in sales over the past year, indicating a rising trend in health-conscious choices."
in_10 = "Analysis of competitor pricing strategies reveals that Grocery Chain A consistently offers lower prices on staple products, attracting cost-conscious shoppers."
in_11 = "Our research indicates that urban millennials are the fastest-growing segment of online grocery shoppers, with a 42% increase in the past two years."

in_12 = "I've noticed some safety hazards in the back storage area that need attention. It's crucial to address these issues to ensure the safety of our team and prevent accidents."
in_13 = "Our team has been working exceptionally hard lately, and it would be motivating to receive more recognition for our efforts, perhaps through an 'Employee of the Month' program."
in_14 = "I'd like to see more opportunities for professional development and training. It would benefit both employees and the company as we stay updated on industry trends."
in_15 = "Improving communication between shifts and departments would help streamline operations and reduce misunderstandings. Clearer communication channels are essential."


input_text_lst_news = [
    in_1,
    in_2,
    in_3,
    in_4,
    in_5,
    in_6,
    in_7,
    in_8,
    in_9,
    in_10,
    in_11,
    in_12,
    in_13,
    in_14,
    in_15,
]

In [ ]:
import numpy as np
from vertexai.language_models import TextEmbeddingModel

embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

- Get embeddings for all pieces of text.
- Store them in a 2D NumPy array (one row for each embedding).

In [ ]:
embeddings = []
for input_text in input_text_lst_news:
    emb = embedding_model.get_embeddings([input_text])[0].values
    embeddings.append(emb)

embeddings_array = np.array(embeddings)

In [ ]:
print("Shape: " + str(embeddings_array.shape))
print(embeddings_array)

#### Reduce embeddings from 768 to 2 dimensions for visualization
- We'll use principal component analysis (PCA).
- You can learn more about PCA in [this video](https://www.coursera.org/learn/unsupervised-learning-recommenders-reinforcement-learning/lecture/73zWO/reducing-the-number-of-features-optional) from the Machine Learning Specialization. 

In [ ]:
from sklearn.decomposition import PCA

# Perform PCA for 2D visualization
PCA_model = PCA(n_components=2)
PCA_model.fit(embeddings_array)
new_values = PCA_model.transform(embeddings_array)

In [ ]:
import pandas as pd
text = pd.DataFrame(input_text_lst_news, columns=["text"])
text

In [ ]:
list(text.columns)

In [ ]:
from utils import umap_plot
umap_plot(emb=embeddings_array, text=text)

In [ ]:
print("Shape: " + str(new_values.shape))
print(new_values)

In [ ]:
import matplotlib.pyplot as plt
import mplcursors
%matplotlib ipympl

from utils import plot_2D
plot_2D(new_values[:,0], new_values[:,1], input_text_lst_news)

#### Embeddings and Similarity
- Plot a heat map to compare the embeddings of sentences that are similar and sentences that are dissimilar.

In [ ]:
in_1 = """He couldn’t desert 
          his post at the power plant."""

in_2 = """The power plant needed 
          him at the time."""

in_3 = """Cacti are able to 
          withstand dry environments."""

in_4 = """Desert plants can 
          survive droughts."""

input_text_lst_sim = [in_1, in_2, in_3, in_4]

In [ ]:
embeddings = []
for input_text in input_text_lst_sim:
    emb = embedding_model.get_embeddings([input_text])[0].values
    embeddings.append(emb)

embeddings_array = np.array(embeddings)

In [ ]:
from utils import plot_heatmap

y_labels = input_text_lst_sim

# Plot the heatmap
plot_heatmap(embeddings_array, y_labels=y_labels, title="Embeddings Heatmap")

Note: the heat map won't show everything because there are 768 columns to show.  To adjust the heat map with your mouse:
- Hover your mouse over the heat map.  Buttons will appear on the left of the heatmap.  Click on the button that has a vertical and horizontal double arrow (they look like axes).
- Left click and drag to move the heat map left and right.
- Right click and drag up to zoom in.
- Right click and drag down to zoom out.

#### Compute cosine similarity
- The `cosine_similarity` function expects a 2D array, which is why we'll wrap each embedding list inside another list.
- You can verify that sentence 1 and 2 have a higher similarity compared to sentence 1 and 4, even though sentence 1 and 4 both have the words "desert" and "plant".

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def compare(embeddings, idx1, idx2):
    return cosine_similarity([embeddings[idx1]], [embeddings[idx2]])

In [ ]:
print(in_1)
print(in_2)
print(compare(embeddings, 0, 1))

In [ ]:
print(in_1)
print(in_4)
print(compare(embeddings, 0, 3))

# Applications of Embeddings

#### Load Stack Overflow questions and answers from BigQuery
- BigQuery is Google Cloud's serverless data warehouse.
- We'll get the first 500 posts (questions and answers) for each programming language: Python, HTML, R, and CSS.

In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")
print(f"REGION: {os.environ['GOOGLE_APPLICATION_CREDENTIALS']}")

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

In [ ]:
# Path to your service account key file
key_path = '../../.keys/alpine-inkwell-403410-78bfa7dd2500.json' #Path to the json key associated with your service account from google cloud

In [ ]:
# Create credentials object

credentials = Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/cloud-platform'])

if credentials.expired:
    credentials.refresh(Request())

In [ ]:
def run_bq_query(sql):

    # Create BQ client
    bq_client = bigquery.Client(project = PROJECT_ID)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, 
                                         use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, 
                                    job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [ ]:
# define list of programming language tags we want to query

language_list = ["python", "html", "r", "css"]

In [ ]:
so_df = pd.DataFrame()

for language in language_list:
    
    print(f"generating {language} dataframe")
    
    query = f"""
    SELECT
        CONCAT(q.title, q.body) as input_text,
        a.body AS output_text
    FROM
        `bigquery-public-data.stackoverflow.posts_questions` q
    JOIN
        `bigquery-public-data.stackoverflow.posts_answers` a
    ON
        q.accepted_answer_id = a.id
    WHERE 
        q.accepted_answer_id IS NOT NULL AND 
        REGEXP_CONTAINS(q.tags, "{language}") AND
        a.creation_date >= "2020-01-01"
    LIMIT 
        500
    """

    
    language_df = run_bq_query(query)
    language_df["category"] = language
    so_df = pd.concat([so_df, language_df], 
                      ignore_index = True) 

- You can reuse the above code to run your own queries if you are using Google Cloud's BigQuery service.
- In this classroom, if you run into any issues, you can load the same data from a csv file.

#### Generate text embeddings
- To generate embeddings for a dataset of texts, we'll need to group the sentences together in batches and send batches of texts to the model.
- The API currently can take batches of up to 5 pieces of text per API call.

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [ ]:
import time
import numpy as np

In [ ]:
# Generator function to yield batches of sentences

def generate_batches(sentences, batch_size = 5):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

In [ ]:
so_questions = so_df[0:200].input_text.tolist() 
batches = generate_batches(sentences = so_questions)

In [ ]:
batch = next(batches)
len(batch)

#### Get embeddings on a batch of data
- This helper function calls `model.get_embeddings()` on the batch of data, and returns a list containing the embeddings for each text in that batch.

In [ ]:
def encode_texts_to_embeddings(sentences):
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

In [ ]:
batch_embeddings = encode_texts_to_embeddings(batch)

In [ ]:
f"{len(batch_embeddings)} embeddings of size \
{len(batch_embeddings[0])}"

#### Code for getting data on an entire data set
- Most API services have rate limits, so we've provided a helper function (in utils.py) that you could use to wait in-between API calls.
- If the code was not designed to wait in-between API calls, you may not receive embeddings for all batches of text.
- This particular service can handle 20 calls per minute.  In calls per second, that's 20 calls divided by 60 seconds, or `20/60`.

```Python
from utils import encode_text_to_embedding_batched

so_questions = so_df.input_text.tolist()
question_embeddings = encode_text_to_embedding_batched(
                            sentences=so_questions,
                            api_calls_per_second = 20/60, 
                            batch_size = 5)
```

In order to handle limits of this classroom environment, we're not going to run this code to embed all of the data. But you can adapt this code for your own projects and datasets.

#### Load the data from file
- We'll load the stack overflow questions, answers, and category labels (Python, HTML, R, CSS) from a .csv file.
- We'll load the embeddings of the questions (which we've precomputed with batched calls to `model.get_embeddings()`), from a pickle file.

In [ ]:
so_df.shape

#### Code takes ~ 20 minutes to run

In [ ]:
#from utils import encode_text_to_embedding_batched
#
#so_questions = so_df.input_text.tolist()
#question_embeddings = encode_text_to_embedding_batched(
#                            sentences=so_questions,
#                            api_calls_per_second = 20/60, 
#                            batch_size = 5)

In [ ]:
so_df = pd.read_csv('data/so_database_app.csv')
so_df.head()

In [ ]:
import pickle

In [ ]:
with open('data/question_embeddings_app.pkl', 'rb') as file:
    question_embeddings = pickle.load(file)

In [ ]:
print("Shape: " + str(question_embeddings.shape))
print(question_embeddings)

#### Cluster the embeddings of the Stack Overflow questions

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
clustering_dataset = question_embeddings[:1000]

In [ ]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, 
                random_state=0, 
                ).fit(clustering_dataset)

In [ ]:
kmeans_labels = kmeans.labels_

In [ ]:
PCA_model = PCA(n_components=2)
PCA_model.fit(clustering_dataset)
new_values = PCA_model.transform(clustering_dataset)

In [ ]:
import matplotlib.pyplot as plt
import mplcursors
%matplotlib ipympl

In [ ]:
from utils import clusters_2D
clusters_2D(x_values = new_values[:,0], y_values = new_values[:,1], 
            labels = so_df[:1000], kmeans_labels = kmeans_labels)

- Clustering is able to identify two distinct clusters of HTML or Python related questions, without being given the category labels (HTML or Python).

## Anomaly / Outlier detection

- We can add an anomalous piece of text and check if the outlier (anomaly) detection algorithm (Isolation Forest) can identify it as an outlier (anomaly), based on its embedding.

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
input_text = """I am making cookies but don't 
                remember the correct ingredient proportions. 
                I have been unable to find 
                anything on the web."""

In [ ]:
emb = model.get_embeddings([input_text])[0].values

In [ ]:
embeddings_l = question_embeddings.tolist()
embeddings_l.append(emb)

In [ ]:
embeddings_array = np.array(embeddings_l)

In [ ]:
print("Shape: " + str(embeddings_array.shape))
print(embeddings_array)

In [ ]:
# Add the outlier text to the end of the stack overflow dataframe
so_df = pd.read_csv('so_database_app.csv')
new_row = pd.Series([input_text, None, "baking"], 
                    index=so_df.columns)
so_df.loc[len(so_df)+1] = new_row
so_df.tail()

#### Use Isolation Forest to identify potential outliers

- `IsolationForest` classifier will predict `-1` for potential outliers, and `1` for non-outliers.
- You can inspect the rows that were predicted to be potential outliers and verify that the question about baking is predicted to be an outlier.

In [ ]:
clf = IsolationForest(contamination=0.005, 
                      random_state = 2) 

In [ ]:
preds = clf.fit_predict(embeddings_array)

print(f"{len(preds)} predictions. Set of possible values: {set(preds)}")

In [ ]:
so_df.loc[preds == -1]

In [ ]:
pd.set_option('display.max_colwidth', None)
so_df.loc[preds == -1]["input_text"]

#### Remove the outlier about baking

In [ ]:
so_df = so_df.drop(so_df.index[-1])

In [ ]:
so_df

## Classification
- Train a random forest model to classify the category of a Stack Overflow question (as either Python, R, HTML or CSS).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# re-load the dataset from file
so_df = pd.read_csv('so_database_app.csv')
X = question_embeddings
X.shape

In [ ]:
y = so_df['category'].values
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 2)

In [ ]:
clf = RandomForestClassifier(n_estimators=200)

In [ ]:
clf.fit(X_train, y_train)

#### You can check the predictions on a few questions from the test set

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred) # compute accuracy
print("Accuracy:", accuracy)

#### Try out the classifier on some questions

In [ ]:
# choose a number between 0 and 1999
i = 2
label = so_df.loc[i,'category']
question = so_df.loc[i,'input_text']

# get the embedding of this question and predict its category
question_embedding = model.get_embeddings([question])[0].values
pred = clf.predict([question_embedding])

print(f"For question {i}, the prediction is `{pred[0]}`")
print(f"The actual label is `{label}`")
print("The question text is:")
print("-"*50)
print(question)

# Text Generation with Vertex AI

#### Project environment setup

- Load credentials and relevant Python Libraries

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")

### Prompt the model
- We'll import a language model that has been trained to handle a variety of natural language tasks, `text-bison@001`.
- For multi-turn dialogue with a language model, you can use, `chat-bison@001`.

In [ ]:
from vertexai.language_models import TextGenerationModel

In [ ]:
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")

#### Question Answering
- You can ask an open-ended question to the language model.

In [ ]:
prompt = "I'm a high school student. \
Recommend me a programming activity to improve my skills."

In [ ]:
print(generation_model.predict(prompt=prompt).text)

#### Classify and elaborate
- For more predictability of the language model's response, you can also ask the language model to choose among a list of answers and then elaborate on its answer.

In [ ]:
prompt = """I'm a high school student. \
Which of these activities do you suggest and why:
a) learn Python
b) learn Javascript
c) learn Fortran
"""

In [ ]:
print(generation_model.predict(prompt=prompt).text)

#### Extract information and format it as a table

In [ ]:
prompt = """ A bright and promising wildlife biologist \
named Jesse Plank (Amara Patel) is determined to make her \
mark on the world. 
Jesse moves to Texas for what she believes is her dream job, 
only to discover a dark secret that will make \
her question everything. 
In the new lab she quickly befriends the outgoing \
lab tech named Maya Jones (Chloe Nguyen), 
and the lab director Sam Porter (Fredrik Johansson). 
Together the trio work long hours on their research \
in a hope to change the world for good. 
Along the way they meet the comical \
Brenna Ode (Eleanor Garcia) who is a marketing lead \
at the research institute, 
and marine biologist Siri Teller (Freya Johansson).

Extract the characters, their jobs \
and the actors who played them from the above message as a table
"""

In [ ]:
response = generation_model.predict(prompt=prompt)

print(response.text)

- You can copy-paste the text into a markdown cell to see if it displays a table.

| Character | Job | Actor |
|---|---|---|
| Jesse Plank | Wildlife Biologist | Amara Patel |
| Maya Jones | Lab Tech | Chloe Nguyen |
| Sam Porter | Lab Director | Fredrik Johansson |
| Brenna Ode | Marketing Lead | Eleanor Garcia |
| Siri Teller | Marine Biologist | Freya Johansson |

### Adjusting Creativity/Randomness
- You can control the behavior of the language model's decoding strategy by adjusting the temperature, top-k, and top-n parameters.
- For tasks for which you want the model to consistently output the same result for the same input, (such as classification or information extraction), set temperature to zero.
- For tasks where you desire more creativity, such as brainstorming, summarization, choose a higher temperature (up to 1).

In [ ]:
temperature = 0.0

In [ ]:
prompt = "Complete the sentence: \
As I prepared the picture frame, \
I reached into my toolkit to fetch my:"

In [ ]:
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)

In [ ]:
print(f"[temperature = {temperature}]")
print(response.text)

In [ ]:
temperature = 1.0

In [ ]:
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)

In [ ]:
print(f"[temperature = {temperature}]")
print(response.text)

#### Top P
- Top p: sample the minimum set of tokens whose probabilities add up to probability `p` or greater.
- The default value for `top_p` is `0.95`.
- If you want to adjust `top_p` and `top_k` and see different results, remember to set `temperature` to be greater than zero, otherwise the model will always choose the token with the highest probability.

In [ ]:
top_p = 0.2

In [ ]:
prompt = "Write an advertisement for jackets \
that involves blue elephants and avocados."

In [ ]:
response = generation_model.predict(
    prompt=prompt, 
    temperature=0.9, 
    top_p=top_p,
)

In [ ]:
print(f"[top_p = {top_p}]")
print(response.text)

#### Top k
- The default value for `top_k` is `40`.
- You can set `top_k` to values between `1` and `40`.
- The decoding strategy applies `top_k`, then `top_p`, then `temperature` (in that order).

In [ ]:
top_k = 20
top_p = 0.7

In [ ]:
response = generation_model.predict(
    prompt=prompt, 
    temperature=0.9, 
    top_k=top_k,
    top_p=top_p,
)

In [ ]:
print(f"[top_p = {top_p}]")
print(response.text)

# Semantic Search, Building a Q&A System

#### Project environment setup

- Load credentials and relevant Python Libraries

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"REGION: {REGION}")
print(f"APPLICATION_CREDENTIALS: {os.environ['GOOGLE_APPLICATION_CREDENTIALS']}")

## Load Stack Overflow questions and answers from BigQuery

In [ ]:
import pandas as pd

In [ ]:
so_database = pd.read_csv('../L4/data/so_database_app.csv')

In [ ]:
print("Shape: " + str(so_database.shape))
print(so_database)

## Load the question embeddings

In [ ]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [ ]:
import numpy as np
from utils import encode_text_to_embedding_batched

- Here is the code that embeds the text.  You can adapt it for use in your own projects.  
- To save on API calls, we've embedded the text already, so you can load it from the saved file in the next cell.

```Python
# Encode the stack overflow data

so_questions = so_database.input_text.tolist()
question_embeddings = encode_text_to_embedding_batched(
            sentences = so_questions,
            api_calls_per_second = 20/60, 
            batch_size = 5)
```

In [ ]:
import pickle
with open('../L4/data/question_embeddings_app.pkl', 'rb') as file:
      
    # Call load method to deserialze
    question_embeddings = pickle.load(file)
  
    print(question_embeddings)

In [ ]:
so_database['embeddings'] = question_embeddings.tolist()

In [ ]:
so_database

## Semantic Search

When a user asks a question, we can embed their query on the fly and search over all of the Stack Overflow question embeddings to find the most simliar datapoint.

In [ ]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [ ]:
query = ['How to concat dataframes pandas']

In [ ]:
query_embedding = embedding_model.get_embeddings(query)[0].values

In [ ]:
cos_sim_array = cosine_similarity([query_embedding],
                                  list(so_database.embeddings.values))

In [ ]:
cos_sim_array.shape

Once we have a similarity value between our query embedding and each of the database embeddings, we can extract the index with the highest value. This embedding corresponds to the Stack Overflow post that is most similiar to the question "How to concat dataframes pandas".

In [ ]:
index_doc_cosine = np.argmax(cos_sim_array)

In [ ]:
index_doc_distances = distances_argmin([query_embedding], 
                                       list(so_database.embeddings.values))[0]

In [ ]:
so_database.input_text[index_doc_cosine]

In [ ]:
so_database.output_text[index_doc_cosine]

## Question answering with relevant context

Now that we have found the most simliar Stack Overflow question, we can take the corresponding answer and use an LLM to produce a more conversational response.

In [ ]:
from vertexai.language_models import TextGenerationModel

In [ ]:
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")

In [ ]:
context = "Question: " + so_database.input_text[index_doc_cosine] +\
"\n Answer: " + so_database.output_text[index_doc_cosine]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
from IPython.display import Markdown, display

t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)

display(Markdown(response.text))

## When the documents don't provide useful information

Our current workflow returns the most similar question from our embeddings database. But what do we do when that question isn't actually relevant when answering the user query? In other words, we don't have a good match in our database.

In addition to providing a more conversational response, LLMs can help us handle these cases where the most similiar document isn't actually a reasonable answer to the user's query.

In [ ]:
query = ['How to make the perfect lasagna']

In [ ]:
query_embedding = embedding_model.get_embeddings(query)[0].values

In [ ]:
cos_sim_array = cosine_similarity([query_embedding], 
                                  list(so_database.embeddings.values))

In [ ]:
cos_sim_array

In [ ]:
index_doc = np.argmax(cos_sim_array)

In [ ]:
context = so_database.input_text[index_doc] + \
"\n Answer: " + so_database.output_text[index_doc]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, answer with 
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

## Scale with approximate nearest neighbor search

When dealing with a large dataset, computing the similarity between the query and each original embedded document in the database might be too expensive. Instead of doing that, you can use approximate nearest neighbor algorithms that find the most similar documents in a more efficient way.

These algorithms usually work by creating an index for your data, and using that index to find the most similar documents for your queries. In this notebook, we will use ScaNN to demonstrate the benefits of efficient vector similarity search. First, you have to create an index for your embedded dataset.

In [ ]:
import scann
from utils import create_index

#Create index using scann
index = create_index(embedded_dataset = question_embeddings, 
                     num_leaves = 25,
                     num_leaves_to_search = 10,
                     training_sample_size = 2000)

In [ ]:
query = "how to concat dataframes pandas"

In [ ]:
import time 

start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
neighbors, distances = index.search(query_embedding, final_num_neighbors = 1)
end = time.time()

for id, dist in zip(neighbors, distances):
    print(f"[docid:{id}] [{dist}] -- {so_database.input_text[int(id)][:125]}...")

print("Latency (ms):", 1000 * (end - start))

In [ ]:
start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
index_doc = np.argmax(cos_sim_array)
end = time.time()

print(f"[docid:{index_doc}] [{np.max(cos_sim_array)}] -- {so_database.input_text[int(index_doc)][:125]}...")

print("Latency (ms):", 1000 * (end - start))